Принцип работы AES в режиме CTR (Counter Mode) пошагово

### Принцип работы AES в режиме CTR

**Шаг 1: Инициализация**

Перед началом шифрования необходимо инициализировать несколько компонентов:
- **Ключ шифрования:** Секретный ключ, известный только отправителю и получателю. Длина ключа может быть 128, 192, или 256 бит.
- **Счетчик (Counter, CTR):** Значение счетчика, которое изменяется для каждого блока данных. Обычно начинается с инициализирующего вектора (IV), который может быть открытым, но должен быть уникальным для каждой операции шифрования.

**Шаг 2: Генерация ключевого потока**

- **Шифрование счетчика:** Для каждого блока данных AES шифрует текущее значение счетчика с использованием заданного ключа шифрования. Результатом шифрования является блок ключевого потока.
- **Инкремент счетчика:** После шифрования значения счетчика его значение увеличивается (обычно на единицу), чтобы гарантировать уникальность ключевого потока для каждого блока данных.

**Шаг 3: Шифрование данных**

- **XOR данных и ключевого потока:** Данные (плейнтекст) XOR'ятся с ключевым потоком, полученным на предыдущем шаге. Результатом этой операции является зашифрованный текст (шифртекст).

**Шаг 4: Передача данных**

- Зашифрованный текст и IV (или начальное значение счетчика) передаются получателю. IV необходим получателю для генерации того же ключевого потока.

### Расшифрование данных

Процесс расшифрования в режиме CTR аналогичен процессу шифрования и выполняется в обратном порядке. Получатель использует тот же ключ шифрования и IV для генерации того же ключевого потока.

**Шаг 1: Генерация ключевого потока**

Получатель генерирует ключевой поток, шифруя значение счетчика (начиная с IV) с использованием того же ключа шифрования.

**Шаг 2: Расшифрование данных**

Зашифрованный текст (шифртекст) XOR'ится с ключевым потоком для восстановления исходных данных (плейнтекста).

In [1]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

def aes_ctr_encrypt(text, key):
    # Генерируем случайный инициализирующий вектор (IV)
    iv = os.urandom(16)
    print(f"IV: {iv}")
    # Создаем объект шифра
    cipher = Cipher(algorithms.AES(key), modes.CTR(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    # Шифруем данные
    ciphertext = encryptor.update(text) + encryptor.finalize()
    # Возвращаем инициализирующий вектор и шифртекст
    return (iv, ciphertext)

def aes_ctr_decrypt(iv, ciphertext, key):
    # Создаем объект шифра с тем же ключом и IV
    cipher = Cipher(algorithms.AES(key), modes.CTR(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    # Расшифровываем данные
    text = decryptor.update(ciphertext) + decryptor.finalize()
    return text

# Пример использования
key = os.urandom(32)  # AES-256 требует ключ длиной 32 байта
print(f'key: {key}')
text = b"AES Test"
print(f'Текст: {text}')

iv, ciphertext = aes_ctr_encrypt(text, key)
print("Зашифрованный текст:", ciphertext)

decrypted_text = aes_ctr_decrypt(iv, ciphertext, key)
print("Расшифрованный текст:", decrypted_text)

key: b'\xb0\x9ekl\xf2\x17\xdb\x9e\xf0;\xc6(BF\xde\xf3<\xbb5w\xbeeqDWB\x95P&\xda\x8d\x93'
Текст: b'AES Test'
IV: b'\x8c-\xaf\xb0\xe1\x0fO\xee:\x93m\xcb\xfe\xca\xf2\xc4'
Зашифрованный текст: b'\xa8\x0c\xc4W\xec\x84\x9bt'
Расшифрованный текст: b'AES Test'


Инициализирующий вектор (IV) — это концепт, используемый в криптографии для обеспечения уникальности и непредсказуемости зашифрованных данных даже в случае, если исходные (открытые) тексты идентичны. IV добавляет элемент случайности в процесс шифрования, предотвращая создание одинаковых зашифрованных блоков (шифртекстов) из одинаковых исходных блоков (плейнтекстов) при использовании того же ключа шифрования.

### Как работает IV

При шифровании каждый блок данных или начальный этап процесса шифрования требует уникального IV. Этот IV совмещается с плейнтекстом (например, путем операции XOR) перед применением алгоритма шифрования, или используется в качестве начального значения для генерации ключевого потока в поточных или блочных шифрах в режимах, подобных CTR.

### Вид IV

- **Формат:** IV обычно представляет собой строку байтов, длина которой зависит от блочного шифра и его конфигурации. Например, для AES с размером блока в 128 бит (16 байтов) IV также будет иметь длину 16 байтов.
- **Генерация:** IV не должен быть секретным, в отличие от ключа шифрования, но он должен быть непредсказуемым и уникальным для каждой операции шифрования с использованием того же ключа. В идеале, IV должен генерироваться с помощью криптографически стойкого генератора случайных чисел.

### Пример использования IV

Допустим, вы используете AES в режиме CTR для шифрования двух сообщений с использованием одного и того же ключа. Если для обоих сообщений используется разный IV, результатом шифрования будет два совершенно разных шифртекста, даже если исходные сообщения идентичны. Это значительно увеличивает безопасность, поскольку злоумышленник, наблюдающий за шифртекстами, не сможет узнать, когда одинаковые сообщения были зашифрованы.